In [19]:
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np 
class HFModel(object):
    def __init__(self ,pretrain_model_path, training_args,):
        self.training_args = training_args
        self.trained=False
        self.pretrain_model_path = pretrain_model_path

    def entrenar(self, X, y):
        self.tokenizer = AutoTokenizer.from_pretrained(self.pretrain_model_path+'tokenizer/')
        model = AutoModelForSequenceClassification.from_pretrained(self.pretrain_model_path+'model/', num_labels=2)
        
#         assert all([type(elem)==str for elem in X])
        data = {'sentence':[text for text in X],
                'label': [label for label in y]
               }
        train_data = Dataset.from_dict(data)
        def tokenize_function(example):
            return self.tokenizer(example["sentence"], truncation=True)
        
        tokenized_dataset = train_data.map(tokenize_function, batched=True)
        from transformers import Trainer
        from transformers import TrainingArguments



        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        trainer = Trainer(
            model,
            self.training_args,
            train_dataset=tokenized_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer,
        )
        output = trainer.train()
        self.trainer = trainer
        self.model = model
        self.trained=True
        return output
    
    def get_X(text_list):
        return np.array(text_list)
    
    def predecir(self, X):
        assert self.trained
        data = {'sentence':[text for text in X]
               }
        dataset = Dataset.from_dict(data)

        def tokenize_function(example):
            return self.tokenizer(example["sentence"], truncation=True)
        
        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        predictions = self.trainer.predict(tokenized_dataset)

        
        return np.argmax(predictions.predictions, axis=-1)
    
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer",num_train_epochs=15)

bert_path = '/home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/'


model = HFModel(pretrain_model_path=bert_path, training_args=training_args)
model.entrenar(['bad','good','positive','negative','awful','disgusting','beautiful','nice'],
               [0,1,1,0,0,0,1,1]
              )


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Didn't find file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/tokenizer/added_tokens.json. We won't load it.
loading file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/tokenizer/vocab.txt
loading file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/tokenizer/tokenizer.json
loading file None
loading file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/tokenizer/special_tokens_map.json
loading file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-base-uncased/tokenizer/tokenizer_config.json
loading configuration file /home/ec2-user/SageMaker/mariano/huggingface/pretrained/bert-b

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 8
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15, training_loss=0.4390783627827962, metrics={'train_runtime': 14.023, 'train_samples_per_second': 8.557, 'train_steps_per_second': 1.07, 'total_flos': 184999960800.0, 'train_loss': 0.4390783627827962, 'epoch': 15.0})